# Tutorial #1: APIを用いたeLabFTWのノート作成

<b>Note:</b> ご不明な点がございましたら、lee.jemyung@naist.ac.jp にご連絡ください

## **Step 6:** APIを使ってテーブルのプロパティを修正してみましょう
#### テーブルのフォント、線色、線状、特定のセル強調などの修正を行うサンプルコード

#### Note: _Step 6_ で<span style='color: orange'>新しくお知らせする部分のタイトル</span>はオレンジ色で表示し、その他の部分では _Step 1~5_ と同じ内容となります。

## 前準備
###　「.env」ファイルの作成
「.env」ファイルの作成に関しては[**<ins>こちら</ins>**](https://github.com/naist-eln/eln/blob/main/API/eLabFTW_API_env.ipynb)をご覧ください。<br>
「.env」ファイルがすでに用意されている場合は、そのまま次に進んでください。

### <span style='color: orange'>更新実験ノートとテーブルの指定</span>
修正したい実験ノートのIDをここであらかじめ指定するか、下記の「実験ノートリストの確認」からセクションでIDリストを確認して指定する。</br>
指定しない場合、最新の実験ノートを更新することになる。

In [45]:
# experiment IDを指定します。
# 指定せずに空欄にしておく場合、最新のノートを更新することになります。

experiment_id = '1923'   

# フォント、文字サイズ、文字色、文字位置、テーブル線の太さ、線色、線の形、背景色などを指定します。
#「properties」で「all」に該当する値はすべてのテーブルに適用され、特定の項目が指定されていると、該当項目のテーブルだけが修正されます。

properties = {
    'all':{
        'font-family' : 'Helvetica',
        'font-size' : '14px',
        'color' : 'grey',   # font color
        'text-align' : 'left',
        'border-width' : '1px',
        'border-style' : 'solid',
        'border-color' : 'blue',
        'background-color' : 'white'
    },
    '目的組成':{
        'font-family' : 'Impact',
        'font-size' : '20px',
        'color' : 'white',   # font color
        'text-align' : 'center',
        'border-width' : '3px',
        'border-style' : 'dashed',
        'border-color' : 'green',
        'background-color' : 'orange' 
    }
}


### コード駆動準備
ライブラリ、パッケージ、モジュールの呼び出しと設定

In [14]:
import os
import re
import elabapi_python
from elabapi_python.rest import ApiException
import json
from requests.exceptions import HTTPError
import numpy as np
import pandas as pd
from pandas import json_normalize
from openpyxl import load_workbook as lw
from IPython.display import display, HTML
from bs4 import BeautifulSoup
import requests
import codecs
import smbclient
from smb.SMBConnection import SMBConnection
from dotenv import load_dotenv
import warnings
from urllib3.exceptions import InsecureRequestWarning
from difflib import get_close_matches

# APIクライアントの設定
load_dotenv()
configuration = elabapi_python.Configuration()
configuration.host = os.environ["HOST_URL"]
configuration.api_key['Authorization'] = os.environ["API_KEY"]
configuration.debug = False
configuration.verify_ssl = False

# SSL警告を無効化
warnings.simplefilter('ignore', InsecureRequestWarning)

# 各種のインスタンスを作成
api_client = elabapi_python.ApiClient(configuration)
api_client.set_default_header(header_name = 'Authorization', header_value = configuration.api_key['Authorization'] )
experimentsApi = elabapi_python.ExperimentsApi(api_client)
templatesApi = elabapi_python.ExperimentsTemplatesApi(api_client)
uploadsApi = elabapi_python.UploadsApi(api_client)
itemsApi = elabapi_python.ItemsApi(api_client)

### 実験ノートリストの確認
作成した全実験ノートを出力する。

In [15]:
# 全実験ノートの確認
try:
    # デフォルト設定で実験ノートを取得
    print("デフォルト設定で実験ノートを取得:")
    experimentsList = experimentsApi.read_experiments()
    print(f"Number of experiments: {len(experimentsList)}")
    
    # 必要なフィールドを抽出してリストに格納
    exp_data = []
    for experiment in experimentsList:
        exp_dict = experiment.to_dict()
        exp_data.append({
            '_date': exp_dict.get('_date'),
            'category_title': exp_dict.get('category_title'),
            'category': exp_dict.get('category'),
            'fullname': exp_dict.get('fullname'),
            'id': exp_dict.get('id'),
            'tags': exp_dict.get('tags'),
            'tags_id': exp_dict.get('tags_id'),
            'up_item_id': exp_dict.get('up_item_id')
        })

    # データフレームに変換
    experiments_df = pd.DataFrame(exp_data)
    experiments_df['_date'] = pd.to_datetime(experiments_df['_date'])   # _date列をdatetime型に変換
    experiments_df = experiments_df.sort_values(by='_date', ascending=False)    # 作成日が新しい順にソート
    print(experiments_df)
    
except ApiException as e:
    print("ExperimentsApiを呼び出す際の例外: %s\n" % e)

デフォルト設定で実験ノートを取得:
Number of experiments: 24
        _date category_title category        fullname    id  \
1  2025-01-27           None     None     jemyung lee  2321   
3  2024-12-19           None     None  Shogo Takasuka  2153   
4  2024-12-19           None     None  Shogo Takasuka  2152   
5  2024-12-19           None     None  Shogo Takasuka  2151   
6  2024-12-19           None     None  Shogo Takasuka  2150   
7  2024-12-19           None     None  Shogo Takasuka  2149   
2  2024-12-06           None     None     jemyung lee  2062   
0  2024-11-18           None     None     jemyung lee  1987   
8  2024-11-18           None     None  Shogo Takasuka  1992   
9  2024-11-11           None     None     jemyung lee  1923   
10 2024-11-08           None     None  Shogo Takasuka  1910   
11 2024-11-08           None     None  Shogo Takasuka  1907   
12 2024-11-07           None     None     jemyung lee  1906   
13 2024-11-06           None     None     jemyung lee  1900   
14 2024-10-

### <span style='color: orange'>実験ノートIDの指定</span>
実験ノートリストを確認し、修正するノートのIDを入力する。 </br>
最新のノートを修正する場合は空欄にしておいてもよい。

In [4]:
if not experiment_id:
    experiment_id = ''    # <- ここに修正するノートのIDを記入。 空欄にしておくと最新のノートを修正することになる

### <span style='color: orange'>実験ノートを取得</span>
実験ノートリストから実験ノートを取得する。</br>
指定されたノートIDがある場合は当該ノート情報を取得し、指定されたIDがない場合は最新ノートの情報を取得する。</br>
取得した実験ノートのIDが表示されます。

In [16]:
try:
    experiments_list = experimentsApi.read_experiments()
    print(f"Number of experiments: {len(experiments_list)}")
    
    if experiment_id:
        exp_lst = [x.id for x in experiments_list]
        if not experiments_list:
            raise ValueError(f"There's no experiment, number of experiments: {len(experiments_list)}")
        elif experiment_id in exp_lst:
            print(f"Selected experiment ID: {experiment_id}")
        else:
            raise ValueError(f"{experiment_id} does not exist in the experiments list")
    else:
        # 最も新しい実験ノートを取得
        if experiments_list:
            experiments_list = sorted(experiments_list, key=lambda x: x._date, reverse=True)  # 作成日でソート
            experiment_id = experiments_list[0].id  # 最新の実験ノートのIDを取得
            print(f"Latest experiment ID: {experiment_id}")        
        else:
            print("No experiments found.")
            exit()

except ApiException as e:
    print(f"Exception when calling ExperimentsApi->read_experiments: {e}\n")
    exit()

Number of experiments: 24
Latest experiment ID: 2321


### <span style='color: orange'>ノート更新の準備</span>
テキストだけを抽出する関数を定義

In [17]:
# Remove special characters from the string 'str'
# Can define more removing character(s) by adding it at 'char'.
def removeSpChar(str, char):
    try:
        rv_str = re.sub(r'[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', '', str)
        if char: rv_str = rv_str.lstrip(char).strip()
        return rv_str
    except Exception as e:
        print(f"An error occurred: {e}\n")

### <span style='color: orange'>テーブルプロパティの修正</span>
テーブルからデータが記録されている 3 番目の列からの形式を修正します。<br>
フォント、文字サイズ、文字色、文字位置、テーブル線の太さ、線色、線の形、背景色などを更新するコードです。<br>
「properties」で「all」に該当する値はすべてのテーブルに適用され、特定の項目が指定されていると、該当項目のテーブルだけが修正されます。

In [46]:
try:
    experiment = experimentsApi.get_experiment(experiment_id)
    exp_body = experiment.body
    exp_title = experiment.title

    # BeautifulSoupを使ってHTMLを解析
    soup = BeautifulSoup(exp_body, 'html.parser')
    
    cnt_soup = soup.find_all(['h1','p'])
    cnt_dict = dict()
    for cs in cnt_soup:
        cst = removeSpChar(cs.text, 'TS')
        if cst:
            cnt_dict[cst] = cs

    tbl_list = list(properties.keys())

    for tbk in tbl_list:
        default_style = ''
        for stl_tag in properties[tbk].keys():
            default_style += f"{stl_tag}: {properties[tbk][stl_tag]}; "
        
        if tbk.lower() == 'all' and tbk not in cnt_dict:
            for tables in soup.find_all('table'):
                for rows in tables.find_all('tr')[2:]:
                    for tag in rows.find_all():
                        tag['style'] = default_style
        elif tbk in cnt_dict:
            cnt_soup = cnt_dict[tbk]
            table = cnt_soup.find_next_sibling('table')

            if table:
                for rows in table.find_all('tr')[2:]:
                    for tag in rows.find_all():
                        tag['style'] = default_style
            else:
                raise ValueError(f"Section '{tbk}' has no table.")
        else:
            raise ValueError(f"Section '{tbk}' does not exist.")

    # 更新されたHTMLを文字列に戻す
    rev_exp_body = str(soup)

    # 新しい内容をもとに既存の内容を更新
    update_body = {
        "body": rev_exp_body,  # 更新されたHTML
    }

    # 実験ノートの内容を更新
    update_response = experimentsApi.patch_experiment(experiment_id, body=update_body)
    print(f"Experiment {experiment_id} updated successfully.")
    # print(update_response)
except ApiException as e:
    print(f"Exception when calling ExperimentsApi->patch_experiment: {e}\n")
    if e.body:
        print(f"Error details: {e.body}\n")
except Exception as e:
    print(f"An error occurred: {e}\n")

Experiment 1923 updated successfully.


お疲れ様でした。<br>
更新した電子ラボ ノートを確認してみましょう。